# install

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
!pip3 install swifter

# Data Load

In [4]:
import sys
sys.path.append('/content/drive/MyDrive/Hackathon/src/preprocessing')
print(sys.path)

['/content', '/env/python', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.8/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/Hackathon/src/preprocessing']


In [5]:
!ls /content/drive/MyDrive/Hackathon/src/preprocessing

config.py      outputs		    __pycache__
data_merge.py  preprocessing.ipynb  tokenizer_BPE.py


In [11]:
import config
import json
import pandas as pd

In [12]:
# json
def load_jsonl(path):
    """
    Read list of objects from a JSON lines file.
    """
    data = []
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.rstrip('\n|\r')))
    data = pd.DataFrame(data)
    print("Loaded {} records from {}".format(len(data), path))
    return data


In [13]:
df_train = load_jsonl(config.train_data)
df_val = load_jsonl(config.val_data)
df_test = load_jsonl(config.test_data)

Loaded 342390 records from drive/MyDrive/Hackathon/data/train.json
Loaded 10942 records from drive/MyDrive/Hackathon/data/val.json
Loaded 4494 records from drive/MyDrive/Hackathon/data/test.json


# Preprocessing

##### 민감정보 마스킹

##### 공백 여러개 제거

In [28]:
import re
import swifter

In [29]:
df_train['utterance'] = df_train.utterance.swifter.apply(lambda x: re.sub(' +', ' ', str(x)))

Pandas Apply:   0%|          | 0/342390 [00:00<?, ?it/s]

In [39]:
df_val['utterence'] = df_val.utterance.swifter.apply(lambda x: re.sub(' +', ' ', str(x)))

Pandas Apply:   0%|          | 0/10942 [00:00<?, ?it/s]

In [40]:
df_test['utterence'] = df_test.utterance.swifter.apply(lambda x: re.sub(' +', ' ', str(x)))

Pandas Apply:   0%|          | 0/4494 [00:00<?, ?it/s]

##### 한명이 여러번 말한 경우 합치기

In [52]:
data = ['']

for i in df_train.iterrows():
  if data[0 - i[0]] == i[1]['speaker']:
    print("check")
    print(f"index : {i[0]}")
    print(data)
    print(data[0 - i[0]])
    print(i[1]['speaker'])
    
  else:
    data.append(i[1]['speaker'])


check
index : 2
['', 'speaker1', 'speaker2']
speaker1
speaker1


In [51]:
df_train.head()

,speaker,personaID,utterance,summary,terminate,conv_no,utterence
0,speaker1,21837,처음뵙겠습니다. 저는 20대 여성입니다,나는 20대 여성이다.,false,1,처음뵙겠습니다. 저는 20대 여성입니다
1,speaker2,30852,반갑습니다. 저는 30대 남자입니다.,나는 30대 남자이다,false,1,반갑습니다. 저는 30대 남자입니다.
2,speaker1,21837,저는 통역가입니다. 하시는일이 있으신가요?,나는 통역가이다.,false,1,저는 통역가입니다. 하시는일이 있으신가요?
3,speaker2,30852,저는 개발자로 일하고 있습니다.,나는 개발자이다,false,1,저는 개발자로 일하고 있습니다.
4,speaker1,21837,그러시군요. 하시는일은 만족하시나요?,,false,1,그러시군요. 하시는일은 만족하시나요?
